In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
%matplotlib inline

#deal with encoding errors:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

import data_prep
from yleana_util import *

In [18]:
FN = 'data/RawStudentDifficultyData.csv'
rawDF = pd.read_csv(FN)
df = data_prep.main(FN)

In [19]:
def makeHTMLTable(df):
    return df.to_html()\
    .replace('<table border="1" class="dataframe">','<table class="table table-striped">') # use bootstrap styling
    
def groupData(df,columns,statVar):
    '''
    group the data by the columns, aggregating the statVar with sum, mean, std
    args:
        df: dataframe
        columns:columns to group by
        statVar: column to aggregate.  
    '''
    subDF = df[columns+[statVar]]
    grouped = subDF.groupby(columns,sort=True)
    groupedDF = grouped.agg([np.size,np.sum, np.mean,np.std])[statVar]
    groupedDF = groupedDF.reset_index()
    return groupedDF

def buildFocusTable(df,studentID,testID,subject,passingThreshold=0.6,minWrong=5,toHTML=True):
    '''
    Get a data frame of topics in which this student is farthest behind the rest of the class, weighted by topic weight.
    These are recommendations for further study
    args:
        df: raw dataframe
        testID: test from which you want to build a recommendation table
        studentID:student ID integer
        subject: math, reading, sentence, or writing
        passingThreshold: minimum score to pass 
        minWrong: minimum number of wrong answers to make a recommendation
        toHTML: convert table to HTML (default True)
    returns:
        rec: Dataframe of topics in which this student is farthest behind the rest of the class,
                ranked by the difference between this student's % correct and the class avg.
    '''
    
    #optionally specify a testID, otherwise use all tests
    if testID is not None:
        df = df.loc[df['testID']==testID,:].copy()
    df = df.loc[df['type']==subject,:]
        
    #list of topics
    topicsDF = getTopicWeight(df)
    
    #get student and class performance on each topic
    studentPerf,classPerf = getClassAvg(df,columns=['studentID','type','topic'],statVar='correct',passingThreshold=passingThreshold)
    classPerf = classPerf.sort('numStudentsGivenTopic')
    q1 = pd.merge(studentPerf,classPerf,on=['type','topic'])
    q2 = pd.merge(q1,topicsDF, how='left',on=['type','topic'])
    rec = q2[['studentID','type','topic','topicWeight','wrong','score','classAvg']].copy()
    rec['scoreDiff']=rec['score'] - rec['classAvg']
    rec['weightedScoreDiff'] = rec['scoreDiff']*rec['topicWeight']
    
    #only recommend areas where the student got at least a few wrong
    rec = rec[rec['wrong']>=minWrong]
    
    for col in ['topicWeight','score','classAvg','scoreDiff']:
        rec[col] = rec[col].round(2)
    rec.sort('weightedScoreDiff',ascending=True, inplace=True)
    rec = rec.loc[rec['studentID']==studentID].head()
    
    if toHTML:
        return makeHTMLTable(rec)
    else:
        return rec

In [20]:
def groupData(df,columns,statVar):
    '''
    group the data by the columns, aggregating the statVar with sum, mean, std
    args:
        df: dataframe
        columns:columns to group by
        statVar: column to aggregate.  
    '''
    subDF = df[columns+[statVar]]
    grouped = subDF.groupby(columns,sort=True)
    groupedDF = grouped.agg([np.size,np.sum, np.mean])[statVar]
    groupedDF = groupedDF.reset_index()
    return groupedDF

def getPerfByColumns(df,columns,statVar):
    '''
    Get score by topic, by student, along with whether the score is above a passing threshold
    args:
        df: clean dataframe
        columns: columns to group by
        statVar: statistic your'e measuring
        passingThreshold: minimum score to pass
    returns: 
        perf: dataframe listing scores by student by topic
    '''
    perf = groupData(df,columns,statVar)
    perf['wrong']=perf['size'] - perf['sum']

    perf.rename(columns={'size':'numQuestions','sum':'numCorrect','mean':'score'},inplace=True)    
    return perf

def buildOpportunityTable(df,studentID,testID,subject,difficulty,toHTML=True):
    '''
    Get a dataframe of topics in which this student got the most wrong answers, 
    optionally specifying a difficulty level.
    args:
        df: raw dataframe
        testID: test from which you want to build a recommendation table
        firstName:first name of student
        subject: math, reading, sentence, or writing
        passingThreshold: minimum score to pass 
        minWrong: minimum number of wrong answers to make a recommendation
        toHTML: convert table to HTML (default True)
    returns:
        rec: Dataframe of topics in which this student is farthest behind the rest of the class,
                ranked by the difference between this student's % correct and the class avg.
    '''
    #optionally specify a testID, otherwise use all tests
    if testID is not None:
        df = df.loc[df['testID']==testID,:]
        
    print difficulty
    if difficulty is not None:
        df = df.loc[df['difficulty']==difficulty,:]
    df = df.loc[df['type']==subject,:]
    df = df.loc[df['studentID'] == studentID,:]

    rec = getPerfByColumns(df,['testID','type','topic'],'correct')
    rec = rec.sort('wrong',ascending=False).head()
    
    if toHTML:
        return makeHTMLTable(rec)
    else:
        return rec

In [48]:
def buildStudentScoreReport(df,studentID,testID):
    studentName = df.loc[df['studentID']==studentID,'firstName'].iloc[0] + '_' + df.loc[df['studentID']==studentID,'lastName'].iloc[0]
    
    #create table dictionaries
    focus = {}
    opportunity = {}
    careless = {}
    
    #Loop through subjects
    for subject in ['math','reading','writing','sentence']:
        focus[subject] = buildFocusTable(df,studentID,testID,subject)
        opportunity[subject] = buildOpportunityTable(df,studentID,testID,subject,difficulty=None)
        careless[subject] = buildOpportunityTable(df,studentID,testID,subject,difficulty='easy')

    html_string = '''
<html>
    <head>
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
        <style>body{ margin:0 100; background:whitesmoke; }</style>
    </head>
    <body>
        <h1>Score Report for '''+ studentName +'''</h1>

        <h2>Math</h2>
        <h3>Focus Concepts</h3>
        <p>Concepts where the student is furthest behind the rest of the class</p>
        '''+focus['math']+'''
        <h3>Opportunity Concepts</h3>
        <p>Concepts where the student got the most wrong answers or blanks</p>
        '''+opportunity['math']+'''
        <h3>Careless Errors</h3>
        <p>"Easy" concepts where the student got the most wrong answers (Not including blanks)</p>
        '''+careless['math']+'''
        <h3>Strengths</h3>
        <p> ... Coming Soon ... </p>
        
        </br>
        
        <h2>Reading</h2>
        <h3>Focus Concepts</h3>
        <p>Concepts where the student is furthest behind the rest of the class</p>
        '''+focus['reading']+'''
        <h3>Opportunity Concepts</h3>
        <p>Concepts where the student got the most wrong answers or blanks</p>
        '''+opportunity['reading']+'''
        <h3>Careless Errors</h3>
        <p>"Easy" concepts where the student got the most wrong answers (Not including blanks)</p>
        '''+careless['reading']+'''
        <h3>Strengths</h3>
        <p> ... Coming Soon ... </p>
        
        </br>
        
        <h2>Writing</h2>
        <h3>Focus Concepts</h3>
        <p>Concepts where the student is furthest behind the rest of the class</p>
        '''+focus['writing']+'''
        <h3>Opportunity Concepts</h3>
        <p>Concepts where the student got the most wrong answers or blanks</p>
        '''+opportunity['writing']+'''
        <h3>Careless Errors</h3>
        <p>"Easy" concepts where the student got the most wrong answers (Not including blanks)</p>
        '''+careless['writing']+'''
        <h3>Strengths</h3>
        <p> ... Coming Soon ... </p>
        
        </br>
        
        <h2>Sentence</h2>
        <h3>Focus Concepts</h3>
        <p>Concepts where the student is furthest behind the rest of the class</p>
        '''+focus['sentence']+'''
        <h3>Opportunity Concepts</h3>
        <p>Concepts where the student got the most wrong answers or blanks</p>
        '''+opportunity['sentence']+'''
        <h3>Careless Errors</h3>
        <p>"Easy" concepts where the student got the most wrong answers (Not including blanks)</p>
        '''+careless['sentence']+'''
        <h3>Strengths</h3>
        <p> ... Coming Soon ... </p>
    </body>
</html>'''
    
    outName = studentName+'_'+str(studentID)+'_'+testID+'.html'
    index = '</br> <a href="http://www.yleana.org/Stats/'+outName+'">'+outName+'</a>'
    with open("reports/index.html", "a") as myfile:
        myfile.write(index)
    
    f = open('reports/'+outName,'w')
    f.write(html_string)
    f.close()

In [49]:
def buildAllStudentReports(df,testID):
    for studentID in df['studentID'].unique():
        buildStudentScoreReport(df,studentID,testID)

buildAllStudentReports(df,testID='OLSAT8.PT2')

In [ ]:
df.loc[df['testID']=='OLSAT8.PT2','studentID'].unique()